In [1]:
import pandas as pd
import seaborn as sns
import statistics
from matplotlib import pyplot as plt
%matplotlib inline

blocks = pd.read_csv("./raw_data/pr/data_block.csv", names=["BLOCK", "NS"])
ecdsa_checks = pd.read_csv("./raw_data/pr/data_ecdsa.csv", names=["BLOCK", "NS"])
shielded_spend = pd.read_csv("./raw_data/pr/data_shieldedSpend.csv", names=["BLOCK", "NS"])
shielded_output = pd.read_csv("./raw_data/pr/data_shieldedOutput.csv", names=["BLOCK", "NS"])


In [2]:
ecdsa_checks.tail()

,BLOCK,NS
18115945,00000000014dbd7e5b4c7b40299c160845c35c66260189...,451
18115946,00000000014dbd7e5b4c7b40299c160845c35c66260189...,385
18115947,00000000014dbd7e5b4c7b40299c160845c35c66260189...,431
18115948,00000000014dbd7e5b4c7b40299c160845c35c66260189...,580
18115949,0000000001e6cfe53b3ac01d73fb52ece9a7498f3ab3aa...,552


In [3]:
def sum_per_block(raw_data, block_index=1, time_index=2):
    block_hash = 0
    summed = []
    data = {}
    for item in raw_data.itertuples():    
        if (block_hash != item[block_index]):     
            # save timing for block
            data[block_hash] = summed
            # new block
            summed = []
            block_hash = item[block_index]       

        # compute till next block
        summed.append(item[time_index])
    
    return pd.DataFrame.from_dict(data, orient='index', columns=["NS"])

In [4]:
block_inputs = sum_per_block(ecdsa_checks)
block_spends = sum_per_block(shielded_spend)
block_outputs = sum_per_block(shielded_output)
block_outputs.tail()

In [31]:
block_inputs.loc['000000000074009a7fae00e0b3350ff3c05ff2c10597398578e361f8bd12b3d4']

NS        4257078
AMOUNT         87
Name: 000000000074009a7fae00e0b3350ff3c05ff2c10597398578e361f8bd12b3d4, dtype: int64

In [40]:
def median_percent_of_block(sum_timings, blocks, block_index=1, time_index=2):
    data = {}
    for item in blocks.itertuples():
        block_hash = item[block_index]
        block_time = item[time_index]
        try:
            item_time = sum_timings.loc[block_hash]['NS']
            median_percent = median(item_time) * 100 / block_time
            data[block_hash] = mean_percent
        except KeyError:
            continue

    return pd.DataFrame.from_dict(data, orient='index', columns=["AVG%"])

In [41]:
avg_input = median_percent_of_block(block_inputs, blocks)
avg_spend = median_percent_of_block(block_spends, blocks)
avg_output = median_percent_of_block(block_outputs, blocks)

NameError: name 'median' is not defined

In [42]:
avg_input.describe()


,AVG%
count,162630.000000
mean,0.003813
std,0.084616
min,0.000002
25%,0.000430
50%,0.001179
75%,0.003770
max,23.551398


In [43]:
avg_spend.describe()

,AVG%
count,1.528380e+05
mean,2.276974e+01
std,4.797780e+01
min,2.160838e-07
25%,5.261371e-04
50%,1.341716e+01
75%,3.356986e+01
max,1.126036e+04


In [44]:
avg_output.describe()

,AVG%
count,152838.000000
mean,32.236983
std,84.369801
min,0.000003
25%,8.036821
50%,22.207397
75%,48.271384
max,8744.125705


In [45]:
data = {}
for item in blocks.itertuples():
    block_hash = item[1]
    block_time = item[2]
    data[block_hash] = 0
    for summed in [block_inputs, block_spends, block_outputs]:
        try:
            item_time = summed.loc[block_hash]['NS']        
            data[block_hash] = data[block_hash] + item_time
        except KeyError:
            continue
    data[block_hash] = data[block_hash] * 100 / block_time
all_summed = pd.DataFrame.from_dict(data, orient='index')

In [47]:
all_summed.describe()

,0
count,427177.000000
mean,25.156087
std,64.887033
min,0.000000
25%,0.000000
50%,0.013796
75%,55.361442
max,11260.434269


In [49]:
all_summed.head()

,0
0000000000529bb02e61819c1c3e9ad7f3ab24935e494a9d767544ad7257c639,0.006214
00000000017e96e16d27fa162ce0d3a655de04f4ff4d1e8e72905fbb3d7ed5c3,0.000000
00000000013a2e8c09557faa525089f174fc664c89e5199162a182c1b508ef9f,0.000000
000000000130dcda27012e0109018b1bc250334ecbe8f16fd2d5ec1bf29ba564,0.000000
0000000000006239e36c30cd32786734778033765675553c498507dc1e575dbb,89.271494
